In [ ]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from util import datasets, methods, models
from benchmark.sensitivity_n import sensitivity_n
import itertools
import numpy as np
from os import path

In [3]:
# General parameters
batch_size = 64
n_batches = 1
n_subsets = 100
data_root = "../data"
use_logits = False
download_dataset = False
dataset_name = "MNIST"
normalize_attrs = True

In [4]:
# CIFAR10 parameters
if dataset_name == "CIFAR10":
    mask_range = list(range(1, 1000, 100))
    dataset = datasets.Cifar(batch_size=batch_size, data_location=path.join(data_root, "CIFAR10"), download=download_dataset, shuffle=False, version="cifar10")
    model = models.CifarResnet(version="resnet32", params_loc=path.join(data_root, "models/cifar10_resnet32.pth"), num_classes=10, output_logits=use_logits)

In [5]:
# MNIST parameters
if dataset_name == "MNIST":
    mask_range = list(range(1, 700, 50))
    dataset = datasets.MNIST(batch_size=batch_size, data_location=path.join(data_root, "MNIST"), download=download_dataset, shuffle=False)
    model = models.MNISTCNN(params_loc=path.join(data_root, "models/mnist_cnn.pth"))

In [6]:
methods = {
    "GuidedGradCAM": methods.GuidedGradCAM(model, model.get_last_conv_layer(), normalize=normalize_attrs),
    "Gradient": methods.Gradient(model, normalize=normalize_attrs),
    "InputXGradient": methods.InputXGradient(model, normalize=normalize_attrs),
    "IntegratedGradients": methods.IntegratedGradients(model, normalize=normalize_attrs),
    "GuidedBackprop": methods.GuidedBackprop(model, normalize=normalize_attrs),
    "Deconvolution": methods.Deconvolution(model, normalize=normalize_attrs),
    "Random": methods.Random(normalize=normalize_attrs)
}

In [7]:
result = sensitivity_n(itertools.islice(dataset.get_test_data(), n_batches), 
                       lambda x: model(x).detach().numpy(), methods, mask_range, mask_value=dataset.mask_value)

Batch 0...


/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/captum/attr/_utils/gradient.py:33: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index
/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:56: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."


Completely 0 attributions returned for sample.


/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [8]:
result["GuidedGradCAM"].shape

(14, 64)